In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
import urllib.request

# 챗봇 데이터 불러오기
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
file_data = pd.read_csv("ChatBotData.csv")

# SBERT
!pip install sentence_transformers
from numpy.linalg import norm # 벡터 사이의 거리 계산
from sentence_transformers import SentenceTransformer

# 사전 학습 모델
model = 'ddobokki/klue-roberta-small-nli-sts'
sbert_model = SentenceTransformer(model)

# 학습 데이터 전처리
np_embed = [file_data.Q.apply(lambda x: sbert_model.encode(x))]                     # 임베딩 벡터 생성
file_data['embedding'] = np_embed[0]                                                # embedding 컬럼 만들기

def cos_sim(A, B):                                                                  # 두 문장 유사도 계산
     return dot(A, B) / (norm(A) *norm(B))

def chatbot(question):
    embedding = sbert_model.encode(question)                                         # 질문 문장 임베딩 벡터
    file_data['score'] = file_data.embedding.apply(lambda x: cos_sim(x, embedding))  # 코사인 유사도 계산
    return file_data.loc[file_data['score'].idxmax()]['A']                           # 높은 유사도 값을 가진 인덱스 추출

# 챗봇 성능 테스트
chatbot('우주에 구멍이 생겼어요')

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# 학습된 모델 저장
embed_path = '/content/drive/MyDrive/NLP/chatbot.npy'
np.save(embed_path, np_embed)

# 학습된 모델 불러오기
embedding_path = '/content/drive/MyDrive/NLP/chatbot.npy'
embedding_vectors = np.load(embedding_path, allow_pickle = True)
